In [1]:
import pandas as pd 
import random
import glob

StatementMeta(, , , Waiting, )

In [2]:
def update_value(value):
    if value>=2:
        return 'High'
    elif value==2:
        return 'Medium'
    else:
        return 'Low'

StatementMeta(, , , Waiting, )

In [1]:
payment_dict={
                'dpd_days_past_due':{
                        'attr':64
                },
                'count_bills_return':{
                                    'high':2,'med':2,'low':1,'attr':65
                                    },
                'count_cheque_return_outward':{
                                    'high':2,'med':2,'low':1,'attr':66
                                    },
                'count_cheque_return_inward':{
                                    'high':2,'med':2,'low':1,'attr':67
                                    },
                'cash_withdrawal_alert':{
                                    'high':2,'med':2,'low':1,'attr':68
                                    },
                'count_lc_development':{
                                     'high':2,'med':2,'low':1,'attr':69
                                    },
                'pay_in_fund_diversion':{
                                     'high':2,'med':2,'low':1,'attr':70
                                    },
                # 'Count_SMA2_Last6months':{
                #                     'high':2,'med':2,'low':1,'attr':71
                #                     },
                # 'Count_SMA1_Last6months':{
                #                      'high':2,'med':2,'low':1,'attr':72
                #                     },
                'pay_out_alert':{
                                     'high':2,'med':2,'low':1,'attr':73
                                    },
                
                }
#read the csv file


df_loan = spark.read.format("delta").load("Tables/loan_master")  
df_loan = df_loan.toPandas()
df_loan = df_loan.drop_duplicates(subset='cust_id', keep="first")


# print(df.columns)
#for financial data
df_write = pd.DataFrame(columns=['cust_id', 'alert_impact', 'alert_type','alert_attribute','alert_date',
                                 'alert_tat','Date_action_taken','alert_status','alert_assgined_to','alert_resolved_by','account_type','industry_type'])


for i in list( payment_dict.keys()):
    #high alert
    df_op = spark.read.format("delta").load(f"Tables/{i}")  
    df_op = df_op.toPandas()  
    # df_op = df_op.toPandas()

    df_op = df_op.drop_duplicates(subset='cust_id', keep="first")

    #merge both the dataframes
    df = df_op.merge(df_loan, on="cust_id", how = 'inner')
    if i=='dpd_days_past_due':
        filtered_values = df.loc[df[i]> 60]
    else:
        filtered_values = df.loc[df[i]> payment_dict[i]['high']]

    df_high = filtered_values[['cust_id','Renewal_Due_Date']].copy()


    df_high['alert_impact']=3
    df_high['alert_type']=3
    df_high['alert_attribute']= payment_dict[i]['attr']
    df_high['alert_date']=pd.to_datetime(df_high['Renewal_Due_Date'],format='mixed')+pd.DateOffset(days=random.choice([1, 4, 8, 10, 3,5,7,8]))
    df_high['alert_tat']= [random.randint(1,60) for _ in range(len(df_high))]
    df_high['Date_action_taken']=pd.to_datetime(df_high['alert_date'])+pd.to_timedelta(df_high['alert_tat'],unit='D' )
    open_prob=0.7
    df_high['alert_status']=['open' if random.random()<open_prob else 'Closed' for _ in range(len(df_high))]
    df_high['alert_assgined_to']= [random.randint(1,20) for _ in range(len(df_high))]
    df_high['alert_resolved_by']= [random.randint(1,20) for _ in range(len(df_high))]
    df_high['account_type']=0
    df_high['industry_type']= 0
    df_high=df_high.drop(['Renewal_Due_Date'], axis=1)
    df_high = df_high.drop_duplicates()

    # df_high.to_csv('alert_file.csv',mode='a')
    df_write = pd.concat([df_write,df_high])


    #medium alert
    if i=='dpd_days_past_due':
        filtered_values = df.loc[df[i]<=60]
        filtered_values = filtered_values.loc[df[i]>30]
    else:
        filtered_values = df.loc[df[i]== payment_dict[i]['med']]
    df_med = filtered_values[['cust_id','Renewal_Due_Date']].copy()


    df_med['alert_impact']=2
    df_med['alert_type']=3
    df_med['alert_attribute']= payment_dict[i]['attr']
    df_med['alert_date']=pd.to_datetime(df_med['Renewal_Due_Date'],format='mixed')+pd.DateOffset(days=random.choice([1, 4, 8, 10, 3,5,7,8]))
    df_med['alert_tat']= [random.randint(1,60) for _ in range(len(df_med))]
    df_med['Date_action_taken']=pd.to_datetime(df_med['alert_date'])+pd.to_timedelta(df_med['alert_tat'],unit='D' )
    open_prob=0.7
    df_med['alert_status']=['open' if random.random()<open_prob else 'Closed' for _ in range(len(df_med))]
    df_med['alert_assgined_to']= [random.randint(1,20) for _ in range(len(df_med))]
    df_med['alert_resolved_by']= [random.randint(1,20) for _ in range(len(df_med))]
    df_med['account_type']=0
    df_med['industry_type']= 0
    df_med=df_med.drop(['Renewal_Due_Date'], axis=1)
    df_med = df_med.drop_duplicates()

    # df_med.to_csv('alert_file.csv',mode='a')
    df_write = pd.concat([df_write,df_med])

    #low alert
    if i=='dpd_days_past_due':
        filtered_values = df.loc[df[i]<30]
    else:
        filtered_values = df.loc[df[i]==payment_dict[i]['low']]
    df_low = filtered_values[['cust_id','Renewal_Due_Date']].copy()


    df_low['alert_impact']=1
    df_low['alert_type']=3
    df_low['alert_attribute']= payment_dict[i]['attr']
    df_low['alert_date']=pd.to_datetime(df_low['Renewal_Due_Date'],format='mixed')+pd.DateOffset(days=random.choice([1, 4, 8, 10, 3,5,7,8]))
    df_low['alert_tat']= [random.randint(1,60) for _ in range(len(df_low))]
    df_low['Date_action_taken']=pd.to_datetime(df_low['alert_date'])+pd.to_timedelta(df_low['alert_tat'],unit='D' )
    open_prob=0.7
    df_low['alert_status']=['open' if random.random()<open_prob else 'Closed' for _ in range(len(df_low))]
    df_low['alert_assgined_to']= [random.randint(1,20) for _ in range(len(df_low))]
    df_low['alert_resolved_by']= [random.randint(1,20) for _ in range(len(df_low))]
    df_low['account_type']=0
    df_low['industry_type']= 0
    df_low=df_low.drop(['Renewal_Due_Date'], axis=1)
    
    df_low = df_low.drop_duplicates()

    # df_low.to_csv('alert_file.csv',mode='a')
    df_write = pd.concat([df_write,df_low])

StatementMeta(, , , Waiting, )

In [4]:
df_write=spark.createDataFrame(df_write)
df_write.write.format("delta").mode("overwrite").save("Tables/alert_file_3")

StatementMeta(, , , Waiting, )